# Week 2, Lab 2: Propositional Logic and Inference

## Welcome Back!

In Lab 1, we learned how to represent knowledge using propositional logic. Now we'll learn how to **reason** with that knowledge - how to derive new facts from existing ones!

### What You'll Learn

- Inference rules (Modus Ponens, Modus Tollens)
- Model checking (brute force reasoning)
- Conjunctive Normal Form (CNF)
- Resolution algorithm
- Forward and backward chaining
- Building an inference engine

### Real-World Applications

- Automated theorem proving
- Expert systems
- Circuit verification
- Planning systems
- Semantic web reasoning

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, combinations
from typing import List, Dict, Set, Tuple, Optional, Union
from copy import deepcopy

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. What is Inference?

**Inference** is the process of deriving new knowledge from existing knowledge.

### Example:

**What we know** (Knowledge Base):
1. "It is raining" (P)
2. "If it is raining, then the ground is wet" (P → Q)

**What we can infer**:
3. "The ground is wet" (Q)

This is called **Modus Ponens**!

### Formal Definition:

- **KB**: Knowledge Base (what we know)
- **α**: Query (what we want to know)
- **KB ⊨ α**: KB entails α (α must be true whenever KB is true)

### Goal:
Given KB, determine if KB ⊨ α (does KB entail α?)

## 2. Inference Rules

Inference rules are patterns that guarantee valid conclusions.

### Modus Ponens
```
P
P → Q
─────
Q
```
"If P is true, and P implies Q, then Q is true"

### Modus Tollens
```
¬Q
P → Q
─────
¬P
```
"If Q is false, and P implies Q, then P is false"

### And-Elimination
```
P ∧ Q
─────
P
```
"If P and Q are both true, then P is true"

### And-Introduction
```
P
Q
─────
P ∧ Q
```
"If P is true and Q is true, then P and Q is true"

In [ ]:
# Implement basic inference rules
class InferenceEngine:
    """Simple inference engine using basic rules."""
    
    def __init__(self):
        self.kb = set()  # Set of known facts
        self.rules = []  # List of (premise, conclusion) rules
    
    def tell(self, fact: str):
        """Add a fact to the knowledge base."""
        self.kb.add(fact)
        print(f"Added to KB: {fact}")
    
    def add_rule(self, premise: str, conclusion: str):
        """Add an implication rule."""
        self.rules.append((premise, conclusion))
        print(f"Added rule: {premise} → {conclusion}")
    
    def modus_ponens(self) -> Set[str]:
        """
        Apply Modus Ponens: If P and (P → Q) are in KB, add Q.
        Returns set of newly inferred facts.
        """
        new_facts = set()
        
        for premise, conclusion in self.rules:
            if premise in self.kb and conclusion not in self.kb:
                new_facts.add(conclusion)
                print(f"  Inferred: {conclusion} (from {premise})")
        
        return new_facts
    
    def forward_chain(self, max_iterations: int = 10):
        """
        Apply forward chaining: repeatedly apply Modus Ponens
        until no new facts can be inferred.
        """
        print("\nForward Chaining:")
        print("=" * 50)
        
        for iteration in range(max_iterations):
            new_facts = self.modus_ponens()
            
            if not new_facts:
                print(f"\nConverged after {iteration} iterations.")
                break
            
            self.kb.update(new_facts)
        
        print(f"\nFinal KB size: {len(self.kb)} facts")
    
    def ask(self, query: str) -> bool:
        """Check if a fact is in the knowledge base."""
        return query in self.kb
    
    def display_kb(self):
        """Display current knowledge base."""
        print("\nKnowledge Base:")
        for fact in sorted(self.kb):
            print(f"  ✓ {fact}")

# Test the inference engine
engine = InferenceEngine()

# Add facts
engine.tell("It is raining")
engine.tell("It is cold")

# Add rules
engine.add_rule("It is raining", "The ground is wet")
engine.add_rule("The ground is wet", "The grass is wet")
engine.add_rule("It is cold", "I need a jacket")

# Perform inference
engine.forward_chain()

# Display results
engine.display_kb()

# Query
print("\nQueries:")
print(f"  Is the grass wet? {engine.ask('The grass is wet')}")
print(f"  Do I need a jacket? {engine.ask('I need a jacket')}")
print(f"  Is it snowing? {engine.ask('It is snowing')}")

## 3. Model Checking

**Model checking** is a brute-force approach to inference:
1. Enumerate all possible truth assignments (models)
2. Check which models satisfy the KB
3. If α is true in all models where KB is true, then KB ⊨ α

### Algorithm:
```
For each possible assignment of truth values:
    If KB is true in this model:
        Check if α is also true
        If α is false in any such model:
            return False
return True
```

### Complexity:
- For n variables: 2^n possible models
- Exponential, but guaranteed to work!

In [ ]:
class PropositionalFormula:
    """Represent a propositional logic formula."""
    
    def __init__(self, formula_type: str, *args):
        """
        Create a formula.
        
        Types:
        - 'symbol': A propositional symbol (e.g., 'P')
        - 'not': Negation of a formula
        - 'and': Conjunction of formulas
        - 'or': Disjunction of formulas
        - 'implies': Implication
        """
        self.type = formula_type
        self.args = args
    
    def evaluate(self, model: Dict[str, bool]) -> bool:
        """Evaluate the formula given a model (truth assignment)."""
        if self.type == 'symbol':
            return model.get(self.args[0], False)
        
        elif self.type == 'not':
            return not self.args[0].evaluate(model)
        
        elif self.type == 'and':
            return all(arg.evaluate(model) for arg in self.args)
        
        elif self.type == 'or':
            return any(arg.evaluate(model) for arg in self.args)
        
        elif self.type == 'implies':
            p, q = self.args
            return (not p.evaluate(model)) or q.evaluate(model)
        
        else:
            raise ValueError(f"Unknown formula type: {self.type}")
    
    def get_symbols(self) -> Set[str]:
        """Get all propositional symbols in the formula."""
        if self.type == 'symbol':
            return {self.args[0]}
        
        symbols = set()
        for arg in self.args:
            if isinstance(arg, PropositionalFormula):
                symbols.update(arg.get_symbols())
        return symbols
    
    def __str__(self):
        if self.type == 'symbol':
            return self.args[0]
        elif self.type == 'not':
            return f"¬{self.args[0]}"
        elif self.type == 'and':
            return f"({' ∧ '.join(str(arg) for arg in self.args)})"
        elif self.type == 'or':
            return f"({' ∨ '.join(str(arg) for arg in self.args)})"
        elif self.type == 'implies':
            return f"({self.args[0]} → {self.args[1]})"
        return "?"

# Helper functions for creating formulas
def Symbol(name: str) -> PropositionalFormula:
    return PropositionalFormula('symbol', name)

def Not(formula: PropositionalFormula) -> PropositionalFormula:
    return PropositionalFormula('not', formula)

def And(*formulas: PropositionalFormula) -> PropositionalFormula:
    return PropositionalFormula('and', *formulas)

def Or(*formulas: PropositionalFormula) -> PropositionalFormula:
    return PropositionalFormula('or', *formulas)

def Implies(p: PropositionalFormula, q: PropositionalFormula) -> PropositionalFormula:
    return PropositionalFormula('implies', p, q)

# Test formula creation
P = Symbol('P')
Q = Symbol('Q')
R = Symbol('R')

# Create formula: (P ∧ Q) → R
formula = Implies(And(P, Q), R)
print(f"Formula: {formula}")

# Test evaluation
model1 = {'P': True, 'Q': True, 'R': True}
model2 = {'P': True, 'Q': True, 'R': False}

print(f"\nModel {model1}:")
print(f"  {formula} = {formula.evaluate(model1)}")

print(f"\nModel {model2}:")
print(f"  {formula} = {formula.evaluate(model2)}")

In [ ]:
def model_check(kb: PropositionalFormula, query: PropositionalFormula) -> bool:
    """
    Check if KB entails query using model checking.
    
    Returns True if query is true in all models where KB is true.
    """
    # Get all symbols in KB and query
    symbols = list(kb.get_symbols() | query.get_symbols())
    
    print(f"Model Checking: Does KB ⊨ {query}?")
    print(f"Symbols: {symbols}")
    print(f"Checking {2**len(symbols)} possible models...\n")
    
    models_checked = 0
    kb_true_count = 0
    counterexample = None
    
    # Try all possible truth assignments
    for values in product([False, True], repeat=len(symbols)):
        model = dict(zip(symbols, values))
        models_checked += 1
        
        # Check if KB is true in this model
        if kb.evaluate(model):
            kb_true_count += 1
            
            # If KB is true but query is false, we found a counterexample
            if not query.evaluate(model):
                counterexample = model
                break
    
    print(f"Results:")
    print(f"  Models checked: {models_checked}")
    print(f"  Models where KB is true: {kb_true_count}")
    
    if counterexample:
        print(f"  \n✗ KB does NOT entail query")
        print(f"  Counterexample: {counterexample}")
        print(f"    KB = {kb.evaluate(counterexample)}")
        print(f"    Query = {query.evaluate(counterexample)}")
        return False
    else:
        print(f"  \n✓ KB ENTAILS query!")
        print(f"  Query is true in all {kb_true_count} models where KB is true.")
        return True

# Example 1: Valid entailment
print("Example 1: (P ∧ (P → Q)) ⊨ Q")
print("=" * 60)
kb1 = And(P, Implies(P, Q))
query1 = Q
result1 = model_check(kb1, query1)

print("\n" + "=" * 60)
print("\nExample 2: (P → Q) ⊨ (¬Q → ¬P)  [Contrapositive]")
print("=" * 60)
kb2 = Implies(P, Q)
query2 = Implies(Not(Q), Not(P))
result2 = model_check(kb2, query2)

## 4. Conjunctive Normal Form (CNF)

**CNF** is a standard form for logical formulas:
- Conjunction (AND) of clauses
- Each clause is a disjunction (OR) of literals
- A literal is a symbol or its negation

### Example:
```
(P ∨ Q) ∧ (¬P ∨ R) ∧ (¬Q ∨ ¬R)
```

### Why CNF?
- Standard form for automated reasoning
- Required for resolution algorithm
- Easy to manipulate

### Conversion Steps:
1. Eliminate implications: P → Q becomes ¬P ∨ Q
2. Move NOT inward (De Morgan's laws)
3. Distribute OR over AND
4. Flatten

In [ ]:
# Simple CNF representation
class Clause:
    """A disjunction of literals."""
    
    def __init__(self, literals: List[Tuple[str, bool]]):
        """
        Create a clause.
        
        Args:
            literals: List of (symbol, is_positive) tuples
                     e.g., [('P', True), ('Q', False)] represents P ∨ ¬Q
        """
        self.literals = set(literals)
    
    def __str__(self):
        if not self.literals:
            return "⊥"  # Empty clause (contradiction)
        
        terms = []
        for symbol, is_positive in sorted(self.literals):
            if is_positive:
                terms.append(symbol)
            else:
                terms.append(f"¬{symbol}")
        return " ∨ ".join(terms)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.literals == other.literals
    
    def __hash__(self):
        return hash(frozenset(self.literals))

# Example clauses
clause1 = Clause([('P', True), ('Q', True)])  # P ∨ Q
clause2 = Clause([('P', False), ('R', True)])  # ¬P ∨ R
clause3 = Clause([('Q', False), ('R', False)])  # ¬Q ∨ ¬R

print("Example CNF formula:")
print(f"  Clause 1: {clause1}")
print(f"  Clause 2: {clause2}")
print(f"  Clause 3: {clause3}")
print(f"\n  Full CNF: ({clause1}) ∧ ({clause2}) ∧ ({clause3})")

## 5. Resolution Algorithm

**Resolution** is a powerful inference rule for CNF formulas.

### Resolution Rule:
```
(P ∨ α)    (¬P ∨ β)
─────────────────────
      (α ∨ β)
```

Where:
- P is a literal
- α and β are clauses

### Example:
```
(P ∨ Q)    (¬P ∨ R)
───────────────────
     (Q ∨ R)
```

### Algorithm for KB ⊨ α:
1. Convert KB ∧ ¬α to CNF
2. Apply resolution repeatedly
3. If we derive the empty clause (⊥), then KB ⊨ α
4. If no new clauses can be derived, then KB ⊭ α

In [ ]:
def resolve(clause1: Clause, clause2: Clause) -> Optional[Clause]:
    """
    Apply resolution to two clauses.
    
    Returns:
        Resolvent clause, or None if resolution not possible
    """
    # Find complementary literals (P in one clause, ¬P in the other)
    for symbol, is_positive in clause1.literals:
        complementary = (symbol, not is_positive)
        
        if complementary in clause2.literals:
            # Found complementary literals! Create resolvent
            new_literals = clause1.literals.copy()
            new_literals.remove((symbol, is_positive))
            new_literals.update(clause2.literals)
            new_literals.remove(complementary)
            
            return Clause(list(new_literals))
    
    return None

# Test resolution
print("Resolution Examples:")
print("=" * 50)

# Example 1: (P ∨ Q) and (¬P ∨ R) → (Q ∨ R)
c1 = Clause([('P', True), ('Q', True)])
c2 = Clause([('P', False), ('R', True)])
resolvent = resolve(c1, c2)

print(f"\nClause 1: {c1}")
print(f"Clause 2: {c2}")
print(f"Resolvent: {resolvent}")

# Example 2: (P) and (¬P) → Empty clause (contradiction!)
c3 = Clause([('P', True)])
c4 = Clause([('P', False)])
resolvent2 = resolve(c3, c4)

print(f"\nClause 1: {c3}")
print(f"Clause 2: {c4}")
print(f"Resolvent: {resolvent2}")
print(f"Empty clause? {len(resolvent2.literals) == 0}")
print("→ This means we found a contradiction!")

In [ ]:
def resolution_algorithm(clauses: Set[Clause]) -> bool:
    """
    Resolution algorithm to check for contradiction.
    
    Returns:
        True if contradiction found (empty clause derived)
        False otherwise
    """
    clauses = set(clauses)  # Make a copy
    new_clauses = set()
    
    print("Resolution Algorithm:")
    print("=" * 60)
    print(f"Starting with {len(clauses)} clauses\n")
    
    iteration = 0
    max_iterations = 20
    
    while iteration < max_iterations:
        iteration += 1
        print(f"Iteration {iteration}:")
        
        # Try all pairs of clauses
        clause_list = list(clauses)
        found_new = False
        
        for i in range(len(clause_list)):
            for j in range(i + 1, len(clause_list)):
                resolvent = resolve(clause_list[i], clause_list[j])
                
                if resolvent is not None:
                    # Check for empty clause (contradiction)
                    if len(resolvent.literals) == 0:
                        print(f"  ✓ Empty clause derived!")
                        print(f"    From: {clause_list[i]}")
                        print(f"      and: {clause_list[j]}")
                        print(f"\n✓ CONTRADICTION FOUND! KB is unsatisfiable.")
                        return True
                    
                    # Add new clause if not already known
                    if resolvent not in clauses:
                        new_clauses.add(resolvent)
                        found_new = True
                        print(f"  New clause: {resolvent}")
                        print(f"    (from {clause_list[i]} and {clause_list[j]})")
        
        if not found_new:
            print(f"  No new clauses derived.")
            print(f"\n✗ No contradiction found. KB is satisfiable.")
            return False
        
        clauses.update(new_clauses)
        new_clauses.clear()
        print(f"  Total clauses: {len(clauses)}\n")
    
    print(f"\n⚠ Reached max iterations ({max_iterations})")
    return False

# Example: Prove (P ∧ (P → Q)) ⊨ Q using resolution
print("Example: Prove (P ∧ (P → Q)) ⊨ Q")
print("Strategy: Show that (P ∧ (P → Q) ∧ ¬Q) is unsatisfiable\n")

# Convert to CNF:
# P ∧ (P → Q) ∧ ¬Q
# = P ∧ (¬P ∨ Q) ∧ ¬Q
clauses_example = {
    Clause([('P', True)]),           # P
    Clause([('P', False), ('Q', True)]),  # ¬P ∨ Q (which is P → Q)
    Clause([('Q', False)])           # ¬Q
}

print("Initial clauses:")
for clause in clauses_example:
    print(f"  {clause}")
print()

result = resolution_algorithm(clauses_example)

## 6. Solving the Knights and Knaves Puzzle with Resolution

Let's use our resolution engine to solve a logic puzzle!

In [ ]:
def solve_knights_knaves_resolution():
    """
    Puzzle: 
    A says: "I am a knave"
    
    Let A_knight = "A is a knight"
    
    If A is a knight (tells truth), then "A is a knave" is true
    → A_knight → ¬A_knight (contradiction!)
    
    So A cannot be a knight. Therefore A is a knave.
    """
    print("Knights and Knaves Puzzle:")
    print("A says: 'I am a knave'")
    print("=" * 60)
    
    # Assume A is a knight and derive contradiction
    print("\nTrying: Assume A is a knight...")
    
    # Clauses:
    # 1. A_knight (assumption)
    # 2. A_knight → ¬A_knight (if A is knight, statement is true)
    #    = ¬A_knight ∨ ¬A_knight = ¬A_knight
    
    clauses = {
        Clause([('A_knight', True)]),     # A is a knight
        Clause([('A_knight', False)])     # A says "I am knave" (¬A_knight)
    }
    
    print("Clauses:")
    for c in clauses:
        print(f"  {c}")
    print()
    
    contradiction = resolution_algorithm(clauses)
    
    if contradiction:
        print("\n→ Assumption leads to contradiction!")
        print("→ Therefore: A is a KNAVE ✓")
    else:
        print("\n→ Assumption is consistent")
        print("→ Therefore: A is a KNIGHT")

solve_knights_knaves_resolution()

## 7. Horn Clauses and Forward Chaining

**Horn Clause**: A clause with **at most one positive literal**

### Types:
1. **Fact**: Single positive literal (e.g., P)
2. **Rule**: One positive, rest negative (e.g., ¬P ∨ ¬Q ∨ R)
   - Equivalent to: (P ∧ Q) → R
3. **Goal**: All negative (e.g., ¬P ∨ ¬Q)

### Why Horn Clauses?
- Efficient inference (linear time!)
- Common in expert systems
- Basis of Prolog

### Forward Chaining:
Start with known facts, apply rules to infer new facts.

In [ ]:
class HornClauseKB:
    """Knowledge base for Horn clauses with forward chaining."""
    
    def __init__(self):
        self.facts = set()  # Known facts
        self.rules = []     # Rules: (premises, conclusion)
    
    def add_fact(self, fact: str):
        """Add a fact."""
        self.facts.add(fact)
    
    def add_rule(self, premises: List[str], conclusion: str):
        """Add a Horn clause rule."""
        self.rules.append((premises, conclusion))
    
    def forward_chain(self, query: str, verbose: bool = True) -> bool:
        """
        Forward chaining to determine if query can be inferred.
        """
        if verbose:
            print(f"\nForward Chaining: Can we infer '{query}'?")
            print("=" * 60)
        
        inferred = set(self.facts)
        iteration = 0
        
        while True:
            iteration += 1
            new_inferences = []
            
            # Check each rule
            for premises, conclusion in self.rules:
                # If all premises are known and conclusion is new
                if all(p in inferred for p in premises) and conclusion not in inferred:
                    new_inferences.append((premises, conclusion))
                    inferred.add(conclusion)
                    
                    if verbose:
                        print(f"  Iteration {iteration}: Inferred '{conclusion}'")
                        print(f"    From: {' ∧ '.join(premises)}")
                    
                    # Check if we've found the query
                    if conclusion == query:
                        if verbose:
                            print(f"\n✓ FOUND: '{query}' is entailed!")
                        return True
            
            # If no new inferences, we're done
            if not new_inferences:
                if verbose:
                    print(f"\n✗ NOT FOUND: '{query}' cannot be inferred.")
                    print(f"Final inferred facts: {sorted(inferred)}")
                return False
    
    def display(self):
        """Display the knowledge base."""
        print("\nHorn Clause Knowledge Base:")
        print("=" * 60)
        print("\nFacts:")
        for fact in sorted(self.facts):
            print(f"  ✓ {fact}")
        
        print("\nRules:")
        for premises, conclusion in self.rules:
            print(f"  ({' ∧ '.join(premises)}) → {conclusion}")

# Example: Animal classification
kb = HornClauseKB()

# Facts
kb.add_fact("Tweety has feathers")
kb.add_fact("Tweety lays eggs")

# Rules
kb.add_rule(["Tweety has feathers", "Tweety lays eggs"], "Tweety is a bird")
kb.add_rule(["Tweety is a bird"], "Tweety can fly")
kb.add_rule(["Tweety can fly"], "Tweety is an animal")

kb.display()

# Queries
kb.forward_chain("Tweety can fly")
kb.forward_chain("Tweety is an animal")
kb.forward_chain("Tweety is a mammal")

## 8. Key Takeaways

### Inference Methods:

1. **Model Checking**
   - Pros: Complete, always works
   - Cons: Exponential (2^n models)
   - Use: Small knowledge bases

2. **Resolution**
   - Pros: Complete, sound, efficient
   - Cons: Requires CNF conversion
   - Use: General purpose theorem proving

3. **Forward Chaining (Horn Clauses)**
   - Pros: Linear time, intuitive
   - Cons: Only works for Horn clauses
   - Use: Expert systems, Prolog

### Complexity:

| Method | Time Complexity | Space |
|--------|----------------|-------|
| Model Checking | O(2^n) | O(n) |
| Resolution | Exp (worst) | Polynomial |
| Forward Chain | O(nk) | O(n) |

Where n = number of symbols, k = number of rules

### When to Use What:

- **Small KB**: Model checking
- **General inference**: Resolution
- **Horn clauses/Rules**: Forward chaining
- **Interactive queries**: Backward chaining (next lab!)

## Next Up

In Lab 3, we'll learn:
- **First-order logic** (predicates, quantifiers)
- **Unification algorithm**
- **More expressive representations**
- **Family trees and relationships**

## Practice Exercises

1. Prove using resolution: ((P → Q) ∧ (Q → R)) ⊨ (P → R)
2. Implement backward chaining for Horn clauses
3. Solve the Wumpus World using propositional logic
4. Convert arbitrary formulas to CNF
5. Build a rule-based expert system for a domain

Excellent work! You can now build reasoning systems! 🧠⚡